In [ ]:
import numpy as np
from tensorflow import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, batch_size=32, dim=(2)):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size

    def __len__(self):
        'Denotes the number of batches per epoch'
        ' Keep simple, lets have 1000 batches per epoch....'
        return 1000

    def __getitem__(self, index):
        'Generate one batch of data'
        
        x=np.random.rand(self.dim,self.batch_size)*2-1
        y_v=np.any([np.all(x>0 ,axis=1),np.all(x<0,axis=1)],axis=0)
        y=(y_v[:,None]!= np.arange(2))

        return x, y_v



In [ ]:
def build_model(hp):
    inputs = keras.Input(shape=(2))
    dense=layers.Dense(units=hp.Int('units',min_value=2, max_value=6, step=1),
                       activation="softmax")(inputs)
    #dense=layers.Dense(4,activation="softmax")(dense)
    outputs= layers.Dense(2,activation="softmax")(dense)

    model=keras.Model(inputs=inputs,outputs=outputs)
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy',
                  metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")])
    return model

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers


labels=['True','False']
batch_size=32000
# Generators
training_generator = DataGenerator((2),batch_size)
validation_generator = DataGenerator((2),batch_size)

inputs = keras.Input(shape=(2))
dense=layers.Dense(4,activation="softmax")(inputs)
#dense=layers.Dense(4,activation="softmax")(dense)
outputs= layers.Dense(2,activation="softmax")(dense)

model=keras.Model(inputs=inputs,outputs=outputs)
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")])

model.summary()

model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6,
                    epochs=10);

In [ ]:
import kerastuner

tuner = kerastuner.tuners.Hyperband(
  build_model,
  objective='val_loss',
  max_epochs=100,
  max_trials=200,
  executions_per_trial=2,
  directory='my_dir')

tuner.search(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6,
                    epochs=10);

tuner.result_summary()